In [4]:
# Load Larger LSTM network and generate text
import sys
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical
# load ascii text and covert to lowercase
filename = "wonderland.txt"
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()
# create mapping of unique chars to integers, and a reverse mapping
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = to_categorical(dataY)
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

Total Characters:  163947
Total Vocab:  65
Total Patterns:  163847


In [ ]:
# define the checkpoint
filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
model.fit(X, y, epochs=50, batch_size=64, callbacks=callbacks_list)

Epoch 1/50
2561/2561 [==============================] - ETA: 0s - loss: 2.8633
Epoch 1: loss improved from inf to 2.86332, saving model to weights-improvement-01-2.8633-bigger.hdf5
2561/2561 [==============================] - 5167s 2s/step - loss: 2.8633
Epoch 2/50
2561/2561 [==============================] - ETA: 0s - loss: 2.5538 
Epoch 2: loss improved from 2.86332 to 2.55384, saving model to weights-improvement-02-2.5538-bigger.hdf5
2561/2561 [==============================] - 26062s 10s/step - loss: 2.5538
Epoch 3/50
2561/2561 [==============================] - ETA: 0s - loss: 2.3774
Epoch 3: loss improved from 2.55384 to 2.37739, saving model to weights-improvement-03-2.3774-bigger.hdf5
2561/2561 [==============================] - 7082s 3s/step - loss: 2.3774
Epoch 4/50
2561/2561 [==============================] - ETA: 0s - loss: 2.2364
Epoch 4: loss improved from 2.37739 to 2.23639, saving model to weights-improvement-04-2.2364-bigger.hdf5
2561/2561 [============================

In [ ]:
# load the network weights
filename = "weights-improvement-XX-X.XXXX-bigger.hdf5" #Replace it with the less loss weights file
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')
# pick a random seed
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
	x = np.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = np.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print("\nDone.")